In [1]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [3]:
###input
ymdb_output_final = '../../../Data/ymdb/ymdb_final_result.xlsx'
target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast8_metabolites_path = '../../../Data/model/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Data/analysis/total_met_inchikey0.pickle'
# uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 'all'
num_process = 5
metabolites_info_to_GEM_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_del_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'rxndb_total_info_to_model_top{num}_{cut_off}_del.csv'

cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/'
rxndb_met_max_score_file = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'rxndb_met_max_score_pd_top{num}_{cut_off}_new.csv'
not_lipid_rxndb_met_max_score_file = f'../../../Results/not_lipid/top50_{cut_off}_add_no_ec_re/' + f'rxndb_met_max_score_pd_top50_0.3.csv'
rxndb_total_info_to_model_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'rxndb_total_info_to_model_top{num}_{cut_off}.csv'
YMDB_success_met_smile_11986_file = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_success_met_smile_1350_file = f'../../../Results/lipid/top{num}_{cut_off}_remain_re/' + f'YMDB_success_met_smile_top{num}_{cut_off}_remain.pickle'
rxndb_to_model_11986_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'rxndb_to_model_top{num}_{cut_off}.csv'
rxndb_to_model_1350_path = f'../../../Results/lipid/top{num}_{cut_off}_re/' + 'rxndb_to_model_with_target_topall_0.3_success.csv'
###output

rxndb_met_max_score_file_output = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/' + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
rxndb_to_model_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}.csv'
rxndb_to_model_12320_plus_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}_12320_plus.csv'
rxndb_to_model_12320_plus2_path = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}_12320_plus2.csv'
YMDB_success_met_smile_file_no_del = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}_no_del.pickle'
YMDB_fail_met_smile_no_del_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}_no_del.pickle'
YMDB_success_met_smile_12320_file = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'YMDB_success_met_smile_top{num}_{cut_off}_new.pickle'
YMDB_success_met_smile_12320_plus_file = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/' + f'YMDB_success_met_smile_top{num}_{cut_off}_plus.pickle'
YMDB_fail_met_smile_12320_plus_file = f'../../../Results/lipid/top{num}_{cut_off}_re_copy_new/' + f'YMDB_fail_met_smile_top{num}_{cut_off}_plus.pickle'
rxndb_to_model_no_del_with_target_new_merge_file = cut_off_path + f'rxndb_to_model_with_target_top{num}_{cut_off}_new_merge.csv'


In [4]:
#check if the output folder exists
if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)


In [4]:
def process_reactions_in_parallel_reactant(rxndb,origin_smile_max_score_pd,sink0_orign, num_processes=5, num_iterations=1):
    num = 0
    tmp_smile_max_score_pd = origin_smile_max_score_pd
    while num < num_iterations:
        num+=1
        sink0 = list(set(sink0_orign))

        # pool = multiprocessing.Pool(num_processes)
        smiles_success = []
        success_rxndbid = []
        chunks = np.array_split(rxndb, 5)
        with multiprocessing.Pool(num_processes) as pool:
            for result in tqdm(pool.imap(partial(process_chunk_reactant,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                sink0_ = sink0 + [smiles2inchikey0(smile) for smile in result[0]]
                sink0_ = list(set(sink0_))
                if len(sink0_) > len(sink0):
                    sink0 = sink0_
                    success_rxndbid.extend(result[1])
                    smiles_success.extend(result[0])

            for result in tqdm(pool.imap(partial(process_chunk_product,smile_max_score_pd=tmp_smile_max_score_pd), chunks), total=len(chunks)):
                sink0_ = sink0 + [smiles2inchikey0(smile) for smile in result[0]]
                sink0_ = list(set(sink0_))
                if len(sink0_) > len(sink0):
                    sink0 = sink0_
                    success_rxndbid.extend(result[1])
                    smiles_success.extend(result[0])
                # smiles_success.extend(result[0])
                # success_rxndbid.extend(result[1])
        # pool.close()
        # pool.join()
        
        smiles_success = list(set(smiles_success))
        success_rxndbid = list(set(success_rxndbid))
        # Process cumulative successful SMILES
        for smile in smiles_success:
            if get_score_from_smiles(smile, tmp_smile_max_score_pd) < 1:
                new_row = {'smile': smile, 'score': 1, 'sim_smile': 'sys'}
                tmp_smile_max_score_pd = tmp_smile_max_score_pd._append(new_row, ignore_index=True)
                # tmp_smile_max_score_pd = tmp_smile_max_score_pd.append(new_row, ignore_index=True)

        # smiles_success = [normalize_smiles(met) for met in smiles_success]
        # smiles_success = list(set(smiles_success))

        print(f'Iteration {num} - Current success count: {len(smiles_success)}')
        print(f'Iteration {num} - Current success_rxndbid count: {len(success_rxndbid)}')
        print('============================================================================')

    print('final success', len(smiles_success))
    print('final success_rxndbid', len(success_rxndbid))
    # print(success_rxndbid)
    return smiles_success, success_rxndbid,tmp_smile_max_score_pd

In [5]:
def process_smiles(i,smile_max_score,yeast_total_smiles):    ###The highest similarity scores of all the reactants and model metabolites of rxn were obtained.
    if i not in smile_max_score['smile']:
        result = {'smile': i, 'score': 0, 'sim_smile': ''}
        score_init = 0
        sim_smile = ' '
        for j in yeast_total_smiles:
            tmp_score = compare_smiles(i, j)  
            # if compare_smiles_inchikey(i,j) or (tmp_score == 1 and are_atom_counts_equal(i,j)):##for the compounds without carbon atom
            if (tmp_score == 1 and are_atom_counts_equal(i,j)):##for the compounds without carbon atom
                score_init = 1
                sim_smile = j
        result['score'] = score_init
        result['sim_smile'] = sim_smile
        return result
    else: return {'smile': '', 'score': '', 'sim_smile': ''}
def calculate_smile_max_score(rxndb_all_reactant_smile,yeast_total_smiles,num_processes=5):
    smile_max_score = {'smile': [], 'score': [], 'sim_smile': []}

    pool = multiprocessing.Pool(num_processes)

    partial_process_smiles = partial(process_smiles, smile_max_score=smile_max_score,yeast_total_smiles=yeast_total_smiles)
    results = list(tqdm(pool.imap(partial_process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))
    # results = list(tqdm(pool.imap(process_smiles, rxndb_all_reactant_smile), total=len(rxndb_all_reactant_smile)))

    pool.close()
    pool.join()

    for result in results:
        smile_max_score['smile'].append(result['smile'])
        smile_max_score['score'].append(result['score'])
        smile_max_score['sim_smile'].append(result['sim_smile'])
        
    return(smile_max_score)
def calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=5):
    smiles_max_score = calculate_smile_max_score(rxndb_all_smiles,yeast8_total_smiles,num_processes=num_processes)
    smiles_max_score_pd = pd.DataFrame(smiles_max_score)
    smiles_max_score_pd.to_csv(rxndb_met_max_score_file,index=None)
    return smiles_max_score_pd

In [6]:
def save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path):
    rxndb_to_model = rxndb_df[rxndb_df['NO'].isin(success_rxndbid)].reset_index(drop=True)
    print(rxndb_to_model.shape)
    rxndb_to_model = rxndb_to_model.drop_duplicates(subset=['templateID', 'rxn_smiles_final'], keep='first').reset_index(drop=True)
    print(rxndb_to_model.shape)
    # rxndb_to_model = rxndb_to_model[['NO','EC_number','templateID','rxn_smiles_basic','rxn_smiles_final']]
    # print(rxndb_to_model.shape)
    rxndb_to_model.to_csv(rxndb_to_model_path,index=None)
    print(rxndb_to_model.shape)

In [8]:
rxndb_total_info_to_model_del = pd.read_csv(rxndb_total_info_to_model_del_path)

In [9]:
rxndb_total_info_to_model_del.shape

(51340, 8)

In [10]:
def find_reaction_containing_metabolite(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['product_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[1].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['product_inchikey0'].progress_apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]
def find_reaction_containing_metabolite_(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['reactant_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[0].split('.')])
    rxndb_to_model_add_no_ec['target_num'] = rxndb_to_model_add_no_ec['reactant_inchikey0'].progress_apply(lambda x: sum(i in inchikey0 for i in x))
    return rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_num'] > 0]

In [7]:
YMDB_success_met_smile_del = load_pickle(YMDB_success_met_smile_12320_file)
YMDB_success_met_inchi_del = [smiles2inchikey0(x) for x in YMDB_success_met_smile_del]
YMDB_success_met_smile_11986 = load_pickle(YMDB_success_met_smile_11986_file)
YMDB_success_met_smile_1350 = load_pickle(YMDB_success_met_smile_1350_file)
YMDB_success_met_smile_13336 = YMDB_success_met_smile_11986 + YMDB_success_met_smile_1350
YMDB_success_met_inchi_13336 = [smiles2inchikey0(x) for x in YMDB_success_met_smile_13336]
diff_inchi = [x for x in YMDB_success_met_inchi_13336 if x not in YMDB_success_met_inchi_del]
len(diff_inchi)

1016

In [12]:
rxndb_to_model_12320 = pd.read_csv(rxndb_total_info_to_model_del_path)
rxndb_to_model_11986 = pd.read_csv(rxndb_to_model_11986_path)
rxndb_to_model_1350 = pd.read_csv(rxndb_to_model_1350_path)

In [13]:
print(rxndb_to_model_11986.shape)
print(rxndb_to_model_1350.shape)
rxndb_to_model_13336 = rxndb_to_model_11986._append(rxndb_to_model_1350,ignore_index=True)
print(rxndb_to_model_13336.shape)

(854584, 5)
(168110, 5)
(1022694, 5)


In [14]:
rxndb_to_model_no_del_with_target = find_reaction_containing_metabolite(inchikey0=diff_inchi, rxndb_to_model_add_no_ec=rxndb_to_model_13336)
print(rxndb_to_model_no_del_with_target.shape)
rxndb_to_model_no_del_with_target = rxndb_to_model_no_del_with_target.drop(['product_inchikey0','target_num'],axis=1)
rxndb_to_model_no_del_with_target_ = find_reaction_containing_metabolite_(diff_inchi, rxndb_to_model_13336)
rxndb_to_model_no_del_with_target_ = rxndb_to_model_no_del_with_target_.drop(['product_inchikey0','reactant_inchikey0','target_num'],axis=1)
print(rxndb_to_model_no_del_with_target_.shape)



  0%|          | 0/1022694 [00:00<?, ?it/s]

100%|██████████| 1022694/1022694 [00:24<00:00, 40912.45it/s]


(105416, 7)


100%|██████████| 1022694/1022694 [00:25<00:00, 40751.16it/s]


(102532, 5)


In [15]:
def find_reaction_containing_metabolite_new(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['product_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[1].split('.')])
    rxndb_to_model_add_no_ec['target_product_inchikey0'] = rxndb_to_model_add_no_ec['product_inchikey0'].progress_apply(lambda x: [i for i in x if i in inchikey0])
    rxndb_to_model_add_no_ec = rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_product_inchikey0'].progress_apply(lambda x: len(x) > 0)]
    rxndb_to_model_add_no_ec['target_product_inchikey0'] = rxndb_to_model_add_no_ec['target_product_inchikey0'].progress_apply(lambda x: sorted(x))
    rxndb_to_model_add_no_ec = rxndb_to_model_add_no_ec.drop_duplicates(subset=['target_product_inchikey0'], keep='first').reset_index(drop=True)
    return rxndb_to_model_add_no_ec
def find_reaction_containing_metabolite_new_(inchikey0, rxndb_to_model_add_no_ec):
    rxndb_to_model_add_no_ec['reactant_inchikey0'] = rxndb_to_model_add_no_ec['rxn_smiles_basic'].progress_apply(lambda x: [smiles2inchikey0(i) for i in x.split('>>')[0].split('.')])
    rxndb_to_model_add_no_ec['target_reactant_inchikey0'] = rxndb_to_model_add_no_ec['reactant_inchikey0'].progress_apply(lambda x: [i for i in x if i in inchikey0])
    rxndb_to_model_add_no_ec = rxndb_to_model_add_no_ec[rxndb_to_model_add_no_ec['target_reactant_inchikey0'].progress_apply(lambda x: len(x) > 0)]
    rxndb_to_model_add_no_ec['target_reactant_inchikey0'] = rxndb_to_model_add_no_ec['target_reactant_inchikey0'].progress_apply(lambda x: sorted(x))
    rxndb_to_model_add_no_ec = rxndb_to_model_add_no_ec.drop_duplicates(subset=['target_reactant_inchikey0'], keep='first').reset_index(drop=True)
    
    
    return rxndb_to_model_add_no_ec

In [16]:
rxndb_to_model_no_del_with_target_new = find_reaction_containing_metabolite_new(inchikey0=diff_inchi, rxndb_to_model_add_no_ec=rxndb_to_model_no_del_with_target)
print(rxndb_to_model_no_del_with_target_new.shape)



100%|██████████| 105416/105416 [00:00<00:00, 1094306.18it/s]


(1150, 7)


In [17]:
rxndb_to_model_no_del_with_target_new_ = find_reaction_containing_metabolite_new_(inchikey0=diff_inchi, rxndb_to_model_add_no_ec=rxndb_to_model_no_del_with_target_)
print(rxndb_to_model_no_del_with_target_new_.shape)



100%|██████████| 102532/102532 [00:00<00:00, 1060258.47it/s]


(962, 7)


In [19]:
rxndb_to_model_no_del_with_target_new = rxndb_to_model_no_del_with_target_new.drop(['product_inchikey0','target_product_inchikey0'],axis=1)

rxndb_to_model_no_del_with_target_new_ = rxndb_to_model_no_del_with_target_new_.drop(['reactant_inchikey0','target_reactant_inchikey0'],axis=1)


In [29]:
rxndb_to_model_no_del_with_target_new_merge = pd.concat([rxndb_to_model_no_del_with_target_new,
                       rxndb_to_model_no_del_with_target_new_],axis=0)
rxndb_to_model_no_del_with_target_new_merge.shape

(2112, 5)

In [30]:
rxndb_to_model_no_del_with_target_new_merge.to_csv(rxndb_to_model_no_del_with_target_new_merge_file,index=None)


In [36]:
rxndb_to_model_no_del_with_target_new_merge = rxndb_to_model_no_del_with_target_new_merge.reset_index(drop=True)
for i in range(len(rxndb_to_model_no_del_with_target_new_merge)):
    if rxndb_to_model_no_del_with_target_new_merge.loc[i, 'NO'] in rxndb_to_model_12320['NO'].tolist():
        print(rxndb_to_model_no_del_with_target_new_merge.loc[i, 'NO'])


In [35]:
rxndb_to_model_diff = rxndb_to_model_no_del_with_target_new_merge[~rxndb_to_model_no_del_with_target_new_merge['rxn_smiles_final'].isin(rxndb_to_model_12320['rxn_smiles_final'])]
rxndb_to_model_diff.shape

(2112, 5)

In [20]:
concat_df = pd.concat([rxndb_to_model_12320,rxndb_to_model_no_del_with_target_new,rxndb_to_model_no_del_with_target_new_],axis=0)
concat_df.shape

(53452, 9)

In [21]:
#get all the metabolites in the rxndb
rxndb_all_smiles = parallel_process_rxn_smiles(concat_df, n_splits=100,num_process=num_process)
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
print(len(yeast8_total_smiles))
rxndb_met_max_score = pd.read_csv(rxndb_met_max_score_file)
sink_smiles = rxndb_met_max_score[rxndb_met_max_score['score'] == 1]['smile'].tolist()
not_lipid_rxndb_met_max_score = pd.read_csv(not_lipid_rxndb_met_max_score_file)
sink_smiles_not_lipid = not_lipid_rxndb_met_max_score[not_lipid_rxndb_met_max_score['score'] == 1]['smile'].tolist()
yeast8_total_smiles = yeast8_total_smiles + sink_smiles  + sink_smiles_not_lipid
yeast8_total_smiles = list(set(yeast8_total_smiles))
print(len(yeast8_total_smiles))


100%|██████████| 100/100 [00:00<00:00, 652.75it/s]


number of metabolites in RXNDB: 30120
945
34085


In [22]:
yeast8_total_smiles_inchi = [smiles2inchikey0(x) for x in yeast8_total_smiles]
rxndb_all_smiles_not_in_yeast8 = [x for x in rxndb_all_smiles if smiles2inchikey0(x) not in yeast8_total_smiles_inchi]
rxndb_all_smiles_in_yeast8 = [x for x in rxndb_all_smiles if x not in rxndb_all_smiles_not_in_yeast8]
print(len(rxndb_all_smiles_in_yeast8))
print(len(rxndb_all_smiles_not_in_yeast8))

29078
1042


In [23]:
#calculate the similarity of the metabolites
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles_not_in_yeast8, yeast8_total_smiles,rxndb_met_max_score_file_output,num_processes=num_process)

100%|██████████| 1042/1042 [27:44<00:00,  1.60s/it]


In [24]:
smile_max_score_pd_ = {'smile': [], 'score': [], 'sim_smile': []}
smile_max_score_pd_['smile'] = rxndb_all_smiles_in_yeast8
smile_max_score_pd_['score'] = 1
smile_max_score_pd_['sim_smile'] = 'sys'
smile_max_score_pd_merge = pd.concat([smile_max_score_pd, pd.DataFrame(smile_max_score_pd_)],axis=0)

In [8]:
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
sink0 = [smiles2inchikey0(x) for x in yeast8_total_smiles]

In [26]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(concat_df, smile_max_score_pd_merge,sink0, num_processes=50, num_iterations=3)


100%|██████████| 100/100 [01:00<00:00,  1.64it/s]


Iteration 1 - Current success count: 26946
Iteration 1 - Current success_rxndbid count: 48959


100%|██████████| 100/100 [01:00<00:00,  1.64it/s]


Iteration 2 - Current success count: 27009
Iteration 2 - Current success_rxndbid count: 49030


100%|██████████| 100/100 [01:02<00:00,  1.60it/s]


Iteration 3 - Current success count: 27029
Iteration 3 - Current success_rxndbid count: 49057
final success 27029
final success_rxndbid 49057


In [28]:
smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(concat_df, smiles_max_score_tmp_pd,sink0, num_processes=50, num_iterations=2)


100%|██████████| 100/100 [01:03<00:00,  1.58it/s]


Iteration 1 - Current success count: 27031
Iteration 1 - Current success_rxndbid count: 49059


100%|██████████| 100/100 [01:03<00:00,  1.56it/s]


Iteration 2 - Current success count: 27031
Iteration 2 - Current success_rxndbid count: 49059
final success 27031
final success_rxndbid 49059


In [31]:
save_success_fail_target_smiles(target_smiles_path,smiles_success,YMDB_success_met_smile_12320_plus_file,YMDB_fail_met_smile_12320_plus_file)


success number: 13167
fail number: 1143
